In [ ]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from datetime import datetime, date
from psx import stocks, tickers
from IPython.display import display
import pandas as pd
import investpy
import yfinance as yf
from matplotlib.dates import relativedelta
from apps.environment.models import Stock
from apps.environment.models.stock import StockBuffer
from apps.environment.models.commodity import Commodity
from apps.environment.models.commodity import CommodityBuffer
import pytz
from os import environ
from math import ceil, floor
import psycopg2



In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Assuming tickers is your DataFrame


tickers_column = tickers()
# display(tickers_column)
symbols = list(tickers_column["symbol"])
names = list(tickers_column["name"])
len_syms = len(symbols)
# # print(symbols)
hundred_inds = pd.read_csv('KSE100 indx Comps - stock-exchange-kse-100pakistan.csv').to_dict(orient='records')
# print(hundred_inds)
hun_syms = {}

all_syms = {}
for ind in range(len_syms):
    all_syms[symbols[ind]] = names[ind]

the_index = 1
for hun_ind in hundred_inds:
    if hun_ind["SYMBOL"] in all_syms:
        sym_b = hun_ind["SYMBOL"]
        if sym_b not in hun_syms:
            hun_syms[sym_b] = {
                "index": the_index,
                "name": all_syms[sym_b]
            }
            the_index = the_index + 1

db_stcks = {}
all_stcks = Stock.objects.all()
max_indx = len(all_stcks)
for astck in all_stcks:
    db_stcks[astck.symbol] = astck.symbol

new_stcks_list = []
for sym_bl in hun_syms:
    if sym_bl not in db_stcks:
        sym_data = hun_syms[sym_bl]
        new_indx = sym_data["index"]
        if new_indx<=max_indx:
            new_indx = max_indx+1
        max_indx = max_indx+1
        new_stcks_list.append(
            Stock(
                index=sym_data["index"],
                symbol=sym_bl,
                name=sym_data["name"],
                type=100
            )
        )

Stock.objects.bulk_create(new_stcks_list)

In [ ]:


all_stocks = Stock.objects.all()

stock_syms = {}

for a_stock in all_stocks:
    if a_stock.symbol not in stock_syms:
        stock_syms[a_stock.symbol] = {
            "object": a_stock,
            "snapshots": {},
            "last_date": None
        }

snapshots = StockBuffer.objects.select_related("stock").all()
for snapshot in snapshots:
    snp_stck = snapshot.stock
    snp_stck_symbl = snp_stck.symbol
    if snp_stck_symbl in stock_syms:
        snp_data = stock_syms[snp_stck_symbl]
        stck_snps = snp_data["snapshots"]
        if snapshot.captured_at not in stck_snps:
            stck_snps[snapshot.captured_at.strftime('%Y-%m-%d %H:%M:%S')] = snapshot
            if not snp_data["last_date"]:
                snp_data["last_date"] = snapshot.captured_at.date()

# tst = 1
for stck_sym, stck_data in stock_syms.items():
    new_snapshots = []
    last_date: datetime = stck_data["last_date"]
    if last_date is None:
        start_from = date(2020, 1, 1)
    else:
        start_from = last_date
    print(last_date, stck_sym, start_from)
    end_date = pytz.utc.localize(datetime.now() + relativedelta(days=2)).date()
    data = stocks(stck_sym, start=start_from, end=end_date)
    data_points_len = len(data)
    stck_snpshts = stck_data["snapshots"]
    the_stock: Stock = stck_data["object"]
    last_data_point = None
    for digi in range(data_points_len):
        data_point = data.iloc[digi]
        time_point = data_point.name.to_pydatetime()
        time_point_str = data_point.name.strftime('%Y-%m-%d %H:%M:%S')
        if time_point_str not in stck_snpshts:
            zone_point_time = pytz.utc.localize(time_point)
            nw_price = data_point.Close
            if last_data_point is None:
                change = 0
            else:
                old_price = last_data_point.Close
                if old_price and old_price > 0:
                    change = (nw_price-old_price)/old_price
                else:
                    change = 0
            new_snapshots.append(StockBuffer(
                stock = the_stock,
                captured_at = zone_point_time,
                price_snapshot = nw_price,
                change = change,
                volume = data_point.Volume,
                bid_vol = 100000000,
                bid_price = nw_price,
                offer_vol = 100000000,
                offer_price = nw_price,
                open = data_point.Open,
                close = nw_price,
                high = data_point.High,
                low = data_point.Low
            ))
        last_data_point = data_point
    StockBuffer.objects.bulk_create(new_snapshots)

    # tst = tst + 1
    # if tst == 4:
    #     break

In [ ]:


all_commodities = Commodity.objects.all()

commodity_syms = {}

for a_commodity in all_commodities:
    if a_commodity.symbol not in commodity_syms:
        commodity_syms[a_commodity.symbol] = {
            "object": a_commodity,
            "snapshots": {},
            "last_date": None
        }

snapshots = CommodityBuffer.objects.select_related("commodity").all()
for snapshot in snapshots:
    snp_cmmdty = snapshot.commodity
    snp_cmmdty_symbl = snp_cmmdty.symbol
    if snp_cmmdty_symbl in commodity_syms:
        cmmdty_data = commodity_syms[snp_cmmdty_symbl]
        cmmdty_snps = cmmdty_data["snapshots"]
        if snapshot.captured_at not in cmmdty_snps:
            cmmdty_snps[snapshot.captured_at.strftime('%Y-%m-%d %H:%M:%S')] = snapshot
            if not cmmdty_data["last_date"]:
                cmmdty_data["last_date"] = snapshot.captured_at.date()

# tst = 1
for cmmdty_sym, cmmdty_data in commodity_syms.items():
    new_snapshots = []
    last_date: datetime = cmmdty_data["last_date"]
    if last_date is None:
        start_from = date(2019, 12, 30)
    else:
        start_from = date(2019, 12, 30)
        # start_from = last_date
    print(last_date, cmmdty_sym, start_from)
    end_date = pytz.utc.localize(datetime.now() + relativedelta(days=2)).date()
    data = yf.download(cmmdty_sym, start=start_from, end=end_date)
    data_points_len = len(data)
    cmmdty_snpshts = cmmdty_data["snapshots"]
    the_commodity: Commodity = cmmdty_data["object"]
    last_data_point = None
    for digi in range(data_points_len):
        data_point = data.iloc[digi]
        time_point = data_point.name.to_pydatetime()
        time_point_str = data_point.name.strftime('%Y-%m-%d %H:%M:%S')
        if time_point_str not in cmmdty_snpshts:
            zone_point_time = pytz.utc.localize(time_point)
            nw_price = data_point.Close
            if last_data_point is None:
                change = 0
            else:
                old_price = last_data_point.Close
                if old_price and old_price > 0:
                    change = (nw_price-old_price)/old_price
                else:
                    change = 0
            new_snapshots.append(CommodityBuffer(
                commodity = the_commodity,
                captured_at = zone_point_time,
                price_snapshot = nw_price,
            ))
        last_data_point = data_point
    CommodityBuffer.objects.bulk_create(new_snapshots)

    # tst = tst + 1
    # if tst == 4:
    #     break

In [ ]:
starting_time_step = datetime(year=2020, month=1, day=1)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True
} 

In [ ]:
the_current_time_step = env_config.get("the_current_time_step")
max_epi_len = 50

__last_time_step = the_current_time_step + relativedelta(days=max_epi_len)
the_current_time_step = pytz.utc.localize(datetime.strptime(str(the_current_time_step), '%Y-%m-%d %H:%M:%S'))
__last_time_step = pytz.utc.localize(datetime.strptime(str(__last_time_step), '%Y-%m-%d %H:%M:%S'))
str_time_step = str(the_current_time_step)

str_last_time_step = str(__last_time_step)
db_params = env_config.get("db_params")
db_conn = psycopg2.connect(**db_params)
cursor = db_conn.cursor()

stcks_query = """
    SELECT *
    FROM stocks
"""
cursor.execute(stcks_query)
stck_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stck_df = pd.DataFrame(stck_data, columns=column_names) 
# stck_df = stck_df.to_dict(orient='records')

stcks_buffer_query = f"""
    SELECT stocks_buffers.*, stocks.index
    FROM stocks_buffers 
    JOIN stocks on stocks.id = stocks_buffers.stock_id
    WHERE 
    captured_at >= '{str_time_step}' AND captured_at <= '{str_last_time_step}'
"""
cursor.execute(stcks_buffer_query)
stcks_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
stcks_buffer_df = pd.DataFrame(stcks_buffer_data, columns=column_names)

# stcks_buffer_df = stcks_buffer_df.to_dict(orient='records')

cmmdties_buffer_query = f"""
    SELECT commodities_buffers.*, commodities.index
    FROM commodities_buffers 
    JOIN commodities on commodities.id = commodities_buffers.commodity_id
    WHERE 
    captured_at >= '{str_time_step}' AND captured_at <= '{str_last_time_step}'
"""
cursor.execute(cmmdties_buffer_query)
cmmdties_buffer_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cmmdties_buffer_df = pd.DataFrame(cmmdties_buffer_data, columns=column_names)
# cmmdties_buffer_df = cmmdties_buffer_df.to_dict(orient='records')

cmmdties_query = """
    SELECT *
    FROM commodities
"""
cursor.execute(cmmdties_query)
cmmdties_data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cmmdties_df = pd.DataFrame(cmmdties_data, columns=column_names) 

In [ ]:
stock_buffers_list = list(StockBuffer.objects.all().filter(captured_at__lte = '2020-05-01'))

# Convert the list of StockBuffer instances to a DataFrame
# Convert the list of StockBuffer instances to a DataFrame
df = pd.DataFrame.from_records([stock_buffer.__dict__ for stock_buffer in stock_buffers_list])

# Convert 'captured_at' column to datetime and localize to UTC
df = pd.DataFrame.from_records([stock_buffer.__dict__ for stock_buffer in stock_buffers_list])

# Convert 'captured_at' column to datetime (if not already) and keep it timezone-aware
df['captured_at'] = pd.to_datetime(df['captured_at'])

# Get unique working days present in the data
unique_working_days = df['captured_at'].dt.date.unique()

# Create a DataFrame to store the final result
result_df = pd.DataFrame()

# Iterate through each stock and working day combination
for stock_id in df['stock_id'].unique():
    stock_data = df[df['stock_id'] == stock_id]

    for working_day in unique_working_days:
        working_day_timestamp = pd.Timestamp(working_day, tz='UTC')
        if not any(stock_data['captured_at'].dt.floor('D').eq(working_day_timestamp)):
            # If data is missing for the working day, use the most recent data for that stock
            most_recent_data = stock_data[stock_data['captured_at'] < working_day_timestamp].sort_values(by='captured_at', ascending=False).head(1)
            if not most_recent_data.empty:
                most_recent_data['captured_at'] = working_day_timestamp
                result_df = pd.concat([result_df, most_recent_data], ignore_index=True)

# Drop duplicate rows based on 'stock_id' and 'captured_at'
result_df = result_df.drop_duplicates(subset=['stock_id', 'captured_at'])

# Sort the DataFrame by 'stock_id' and 'captured_at'
result_df = result_df.sort_values(by=['stock_id', 'captured_at'])

# Reset the index
result_df = result_df.reset_index(drop=True)

In [ ]:
display(stcks_buffer_df)

In [ ]:
stcks_buffer_df['captured_at'] = pd.to_datetime(stcks_buffer_df['captured_at'])

# Set 'stock_id' and 'captured_at' as the index
stcks_buffer_df.set_index(['stock_id', 'captured_at'], inplace=True)

# Resample the data to fill in the missing days
resampled_df = stcks_buffer_df.groupby(level='stock_id').resample('D').ffill()

# Reset the index to make 'stock_id' and 'captured_at' regular columns
resampled_df.reset_index(inplace=True)

# Ensure that the 'captured_at' column is in datetime format
stcks_buffer_df['captured_at'] = pd.to_datetime(stcks_buffer_df['captured_at'])

# Create a list to store the final DataFrames
result_datasets = []

# Group by stock_id and iterate over each group
for stock_id, group in stcks_buffer_df.groupby('stock_id'):
    # Sort the group by 'captured_at' to ensure it's ordered
    group = group.sort_values(by='captured_at')

    # Fill missing values with the previous day's values
    group_filled = group.fillna(method='ffill')

    # Fill remaining missing values with the next day's values
    group_filled = group_filled.fillna(method='bfill')

    # Append the filled group to the result_datasets list
    result_datasets.append(group_filled)

# Concatenate the datasets into a single DataFrame
final_stocks_buffer_df = pd.concat(result_datasets)


print(len(final_stocks_buffer_df))
display(final_stocks_buffer_df)


In [102]:
# Sample data for two datasets
import numpy as np
data1 = {'ID': [1, 2, 3, 4],
         'Name': ['Alice', 'Bob', 'Charlie', 'David'],
         'ind': ['a', 'b', 'c', 'd']
         }
df1 = pd.DataFrame(data1)

data2 = {'ID': [1, 2, 3, 4, 5],
         'Name': [np.nan, np.nan, 'x', np.nan, np.nan],
         'ind': ['a', 'b', 'c', 'd', 'b']
         }
df2 = pd.DataFrame(data2)

# Merge based on the 'ID' column
merged_df = pd.merge(df1, df2, on='ID', how="outer", suffixes=('_left', '_right'))
display(merged_df)
merged_df['Name_left'] = merged_df['Name_left'].fillna(merged_df.groupby(['ind_left', 'ind_right'])['Name_left'].ffill().bfill())
display(merged_df)


,ID,Name,ind_left,ind_right
0,1,Alice,a,a
1,2,Bob,b,b
2,3,Charlie,c,c
3,4,David,d,d
4,5,NaN,NaN,b


,ID,Name,ind_left,ind_right
0,1,Alice,a,a
1,2,Bob,b,b
2,3,Charlie,c,c
3,4,David,d,d
4,5,NaN,NaN,b


In [105]:
import pandas as pd
import numpy as np

data1 = {'ID': [1, 2, 3, 4],
         'Name': ['Alice', 'Bob', 'Charlie', 'David'],
         'ind': ['a', 'b', 'c', 'd']}
df1 = pd.DataFrame(data1)

data2 = {'ID': [1, 2, 3, 4, 5],
         'Name': [np.nan, np.nan, np.nan, np.nan, np.nan],
         'ind': ['a', 'b', 'c', 'd', 'b']}
df2 = pd.DataFrame(data2)

# Filter rows where 'Name_left' is NaN
df_left_missing = df1[df1['Name'].isna()]

# Merge all missing rows with 'Name_right' not NaN
merged_df = pd.merge(df_left_missing, df2, on='ID', how='left', suffixes=('_left', '_right'))

# Use 'Name_right' to fill 'Name_left'
merged_df['Name_left'] = merged_df['Name_right']

# Drop unnecessary columns
merged_df = merged_df[['ID', 'Name_left', 'ind_left']]

# Concatenate with the original dataframe, including all missing rows from df2
result_df = pd.concat([df1, merged_df, df2[df2['Name'].notna()]]).sort_values(by='ID').reset_index(drop=True)

print(result_df)

   ID     Name ind  Name_left ind_left
0   1    Alice   a        NaN      NaN
1   2      Bob   b        NaN      NaN
2   3  Charlie   c        NaN      NaN
3   4    David   d        NaN      NaN


In [110]:


# Sample data for illustration
data1 = {'date': ['2023-01-01', '2023-01-02', '2023-01-03'],
         'id': [1, 2, 3],
         'value_left': ['A', 'B', 'C']}
df1 = pd.DataFrame(data1)

data2 = {'date': ['2023-01-01', '2023-01-02', '2023-01-04'],
         'id': [1, 2, 3],
         'value_right': ['X', 'Y', 'Z']}
df2 = pd.DataFrame(data2)

# Merge dataframes based on 'date' and 'id'
total_merge = df1.merge(df2, on='id', how='outer', indicator=True)
dfs = {k:v for k,v in total_merge.groupby('_merge')}
dfs['both']

/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/199389832.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfs = {k:v for k,v in total_merge.groupby('_merge')}


,date_x,id,value_left,date_y,value_right,_merge
0,2023-01-01,1,A,2023-01-01,X,both
1,2023-01-02,2,B,2023-01-02,Y,both
2,2023-01-03,3,C,2023-01-04,Z,both


In [142]:
# Sample data for illustration
data1 = {'date': ['2023-01-01', '2023-01-02', '2023-01-03'],
         'id': [1, 2, 3],
         'value_left': ['A', 'B', 'C']}
df1 = pd.DataFrame(data1)

data2 = {'date': ['2023-01-01', '2023-01-04', '2023-01-03'],
         'id': [1, 2, 3],
         'value_right': ['X',  np.nan, np.nan]}
df2 = pd.DataFrame(data2)

# Merge dataframes on 'date' and 'id' and keep only the rows present in df2
merged_df = pd.merge(df2, df1, on=['date', 'id'], how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
print("merged_df", merged_df)
merged_df =merged_df.drop('value_left', axis=1)
merged_df = merged_df.rename(columns={'value_right': 'value_left'})

print(merged_df)

concatenated_df = pd.concat([df1, merged_df], ignore_index=True)

# Print the concatenated dataframe
print(concatenated_df)

# Sort the DataFrame by 'id' and 'date'
concatenated_df = concatenated_df.sort_values(by=['id', 'date'])

# Forward fill NaN values based on 'id'
concatenated_df['value_left'] = concatenated_df.groupby('id')['value_left'].fillna(method='ffill')

# Backward fill remaining NaN values based on 'id'
concatenated_df['value_left'] = concatenated_df.groupby('id')['value_left'].fillna(method='bfill')

# Display the updated DataFrame
print(concatenated_df)



merged_df          date  id value_right value_left
1  2023-01-04   2         NaN        NaN
         date  id value_left
1  2023-01-04   2        NaN
         date  id value_left
0  2023-01-01   1          A
1  2023-01-02   2          B
2  2023-01-03   3          C
3  2023-01-04   2        NaN
         date  id value_left
0  2023-01-01   1          A
1  2023-01-02   2          B
3  2023-01-04   2          B
2  2023-01-03   3          C


/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/1115710762.py:29: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  concatenated_df['value_left'] = concatenated_df.groupby('id')['value_left'].fillna(method='ffill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/1115710762.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  concatenated_df['value_left'] = concatenated_df.groupby('id')['value_left'].fillna(method='bfill')


In [158]:
# Sample data for illustration
data1 = {'date': ['2023-01-01', '2023-01-02', '2023-01-03'],
         'id': [1, 2, 3],
         'value': ['A', 'B', 'C'],
         'valuea': ['a', 'b', 'c']}
df1 = pd.DataFrame(data1)

data2 = {'date': ['2023-01-01', '2023-01-04', '2023-01-04', '2023-01-03'],
         'id': [1, 1, 2, 3],
         'value': ['X', np.nan,  np.nan, np.nan,],
         'valuea': [np.nan, np.nan, np.nan, np.nan]}
df2 = pd.DataFrame(data2)

# Merge dataframes on 'date' and 'id' and keep only the rows present in df2
merged_df = pd.merge(df2, df1, on=['date', 'id'], how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
merged_df =merged_df.drop(['value_x', "valuea_x", "valuea_y"], axis=1)
# merged_df =merged_df.drop('valuea_x', axis=1)
merged_df = merged_df.rename(columns={'value_y': 'value', "valuea_y": "valuea"})


concatenated_df = pd.concat([df1, merged_df], ignore_index=True)

# Print the concatenated dataframe

# Sort the DataFrame by 'id' and 'date'
concatenated_df = concatenated_df.sort_values(by=['id', 'date'])

# Forward fill NaN values based on 'id'
concatenated_df['value'] = concatenated_df.groupby('id')['value'].fillna(method='ffill')
concatenated_df['value'] = concatenated_df.groupby('id')['value'].fillna(method='bfill')

concatenated_df['valuea'] = concatenated_df.groupby('id')['valuea'].fillna(method='ffill')
concatenated_df['valuea'] = concatenated_df.groupby('id')['valuea'].fillna(method='bfill')

# Display the updated DataFrame
print(concatenated_df)

         date  id value valuea
0  2023-01-01   1     A      a
3  2023-01-04   1     A      a
1  2023-01-02   2     B      b
4  2023-01-04   2     B      b
2  2023-01-03   3     C      c


/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/3868119292.py:29: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  concatenated_df['value'] = concatenated_df.groupby('id')['value'].fillna(method='ffill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/3868119292.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  concatenated_df['value'] = concatenated_df.groupby('id')['value'].fillna(method='bfill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/3868119292.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  concatenated_df['valuea'] = concatenated_df.groupby('id')['valuea'].fillna(method='ffill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/3868119292.p

In [130]:

# Your dataset with additional columns
data = {'date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'],
        'id': [1, 2, 3, 2],
        'value_left': ['A', 'B', 'C', None],
        'other_column1': [10, 20, 30, None],
        'other_column2': [100, 200, 300, None]}

df = pd.DataFrame(data)

# Sort the DataFrame by 'id' and 'date'
df = df.sort_values(by=['id', 'date'])

# Forward fill NaN values based on 'id' for specific columns
columns_to_fill = ['value_left', 'other_column1', 'other_column2']
df[columns_to_fill] = df.groupby('id')[columns_to_fill].fillna(method='ffill')

# Backward fill remaining NaN values based on 'id' for specific columns
df[columns_to_fill] = df.groupby('id')[columns_to_fill].fillna(method='bfill')

# Fill remaining NaN values in other columns with 0
df = df.fillna(0)

# Display the updated DataFrame
print(df)

         date  id value_left  other_column1  other_column2
0  2023-01-01   1          A           10.0          100.0
1  2023-01-02   2          B           20.0          200.0
3  2023-01-04   2          B           20.0          200.0
2  2023-01-03   3          C           30.0          300.0


/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/1853133386.py:15: FutureWarning: DataFrameGroupBy.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[columns_to_fill] = df.groupby('id')[columns_to_fill].fillna(method='ffill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/1853133386.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[columns_to_fill] = df.groupby('id')[columns_to_fill].fillna(method='ffill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/1853133386.py:18: FutureWarning: DataFrameGroupBy.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[columns_to_fill] = df.groupby('id')[columns_to_fill].fillna(method='bfill')
/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_48833/1853133386.py:18: Fut